In [1]:
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.document_loaders import BigQueryLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS



In [2]:
from dotenv import load_dotenv
import os

# Load the environment variables from the .env file
load_dotenv()

# Access environment variables
api_key = os.getenv("OPENAI_KEY")

In [3]:
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo", openai_api_key=api_key)

In [4]:


PROJECT = "wagon-bootcamp-377120"
DATASET = "g_adventures_dataset"
TABLE = "one_month"

BASE_QUERY = query = f"""
    SELECT
      MAX(tour_operator) AS tour_operator,
      tour_name,
      itinerary_name,
      MAX(visited_countries) AS visited_countries,
      MAX(currency) AS currency,
      ARRAY_AGG(DISTINCT Standard___Adult) AS Costs,
      MAX(duration) AS duration,
      ARRAY_AGG(CAST(start_date AS STRING) ORDER BY start_date) AS start_dates,
      MAX(Travel_Style) AS Travel_Style,
      MAX(Service_Level) AS Service_Level,
      MAX(Physical_Grading) AS Physical_Grading,
      MAX(Merchandising) AS Merchandising,
      MAX(Trip_Type) AS Trip_Type,
      MAX(itinerary) AS itinerary,
      MAX(url) AS url
    FROM
      {PROJECT}.{DATASET}.{TABLE}
    WHERE 
      tour_name = 'Highlights of Jordan'
    GROUP BY
      tour_name, itinerary_name;
    """

loader = BigQueryLoader(BASE_QUERY)
docs = loader.load()

In [5]:

text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
texts = text_splitter.split_documents(docs)

In [6]:

embeddings = OpenAIEmbeddings(openai_api_key=api_key)
retriever = FAISS.from_documents(docs, embeddings).as_retriever()

In [7]:

prompt_template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Answer:"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

In [8]:


chain_type_kwargs = {"prompt": PROMPT}
qa = RetrievalQA.from_chain_type(llm=OpenAI(openai_api_key=api_key), chain_type="stuff", retriever=retriever, chain_type_kwargs=chain_type_kwargs)

In [18]:
tools = [
    Tool(
        name="highlights_of_jordan",
        func=qa.run,
        description="Useful when you need to answer questions about the highlights of jordan itinerary.",
    ),
]

In [19]:
mrkl = initialize_agent(
    tools,
    llm,
    agent=AgentType.OPENAI_FUNCTIONS,
    verbose=True,
    max_iterations=5,
    early_stopping_method="generate",
)

In [23]:
mrkl.run("when does the highlights of jordan itinerary depart?")



> Entering new AgentExecutor chain...

Invoking: `highlights_of_jordan` with `departure`


 Any timeThe Highlights of Jordan itinerary can depart at any time.

> Finished chain.


'The Highlights of Jordan itinerary can depart at any time.'